# Next Steps

## Preparation

In [13]:
# Import prism models from the previous step
import stormvogel
import stormpy

In [14]:
prism_program = stormpy.parse_prism_program('examples/orchard_stormvogel.pm')
constants = "NUM_FRUIT=4, DISTANCE_RAVEN=5"
prism_program = stormpy.preprocess_symbolic_input(prism_program, [], constants)[0].as_prism_program()

options = stormpy.BuilderOptions()
options.set_build_state_valuations()
options.set_build_choice_labels()
options.set_build_with_choice_origins()
orchard_prism = stormpy.build_sparse_model_with_options(prism_program, options)

## Configuration

### Model representation

In [3]:
orchard_symbolic = stormpy.build_symbolic_model(prism_program)
print(orchard_symbolic)

-------------------------------------------------------------- 
Model type: 	MDP (symbolic)
States: 	22469 (85 nodes)
Transitions: 	44954 (916 nodes)
Choices: 	29354
Reward Models:  rounds
Variables: 	rows: 7 meta variables (19 DD variables), columns: 7 meta variables (19 DD variables), nondeterminism: 12 meta variables (12 DD variables)
Labels: 	5
   * deadlock -> 0 state(s) (1 nodes)
   * init -> 1 state(s) (20 nodes)
   * PlayersWon
   * RavenOneAway
   * RavenWon
-------------------------------------------------------------- 



In [4]:
formula = stormpy.parse_properties('Pmax=? [F "PlayersWon"]')[0]
symbolic_result = stormpy.model_checking(orchard_symbolic, formula, only_initial_states=True)
filter = stormpy.create_filter_initial_states_symbolic(orchard_symbolic)
symbolic_result.filter(filter)
print("Maximal probability: {}".format(symbolic_result))

WARN  (SymbolicMinMaxLinearEquationSolver.cpp:64): Selected method is not supported for this solver, switching to value iteration.
Maximal probability: 0.631356
WARN  (SymbolicMinMaxLinearEquationSolver.cpp:64): Selected method is not supported for this solver, switching to value iteration.


### Bisimulation

In [7]:
formula = stormpy.parse_properties('Pmax=? [F "PlayersWon"]')
print("Model with {} states and {} transitions".format(orchard_prism.nr_states, orchard_prism.nr_transitions))
orchard_bisim = stormpy.perform_bisimulation(orchard_prism, formula, stormpy.BisimulationType.STRONG)
print("Model with {} states and {} transitions".format(orchard_bisim.nr_states, orchard_bisim.nr_transitions))

Model with 22469 states and 44954 transitions
Model with 956 states and 2446 transitions


### Model checking algorithms

In [9]:
# Change precision
env = stormpy.Environment()
prec = stormpy.Rational(0.1)
env.solver_environment.minmax_solver_environment.precision = prec
result = stormpy.model_checking(orchard_prism, formula[0], environment=env)
print(result.at(orchard_prism.initial_states[0]))

0.5815061686029693


In [10]:
# Change algorithm
import time
methods = [stormpy.MinMaxMethod.value_iteration,
           stormpy.MinMaxMethod.policy_iteration,
           stormpy.MinMaxMethod.optimistic_value_iteration
          ]
for m in methods:
    env = stormpy.Environment()
    env.solver_environment.minmax_solver_environment.method = m
    start = time.time()
    result = stormpy.model_checking(orchard_prism, formula[0],
                                    environment=env, extract_scheduler=True)
    print(f"Method: {m}")
    print(f"Result: {result.at(orchard_prism.initial_states[0])}")
    print(f"Time: {time.time() - start:.2}s")

Method: MinMaxMethod.value_iteration
Result: 0.6313572986959962
Time: 0.012s
WARN  (IterativeMinMaxLinearEquationSolver.cpp:191): Expected VI operator to be initialized for scheduler extraction. Initializing now, but this is inefficient.
Method: MinMaxMethod.policy_iteration
Result: 0.6313573066006366
Time: 0.25s
Method: MinMaxMethod.optimistic_value_iteration
Result: 0.6313576699315776
Time: 0.013s
WARN  (IterativeMinMaxLinearEquationSolver.cpp:191): Expected VI operator to be initialized for scheduler extraction. Initializing now, but this is inefficient.


## Writing an LP-based MDP Model Checker

In [17]:
# Install LP solver
!pip install pyscipopt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.2/16.2 MB 3.1 MB/s eta 0:00:00a 0:00:01


In [18]:
# These are the target states
players_won = stormpy.parse_properties('"PlayersWon"')[0].raw_formula
psi_states = stormpy.model_checking(orchard_prism, players_won).get_truth_values()

# These are the states that can never reach the target states
phi_states = stormpy.BitVector(orchard_prism.nr_states, True)
prob0max_states = stormpy.compute_prob01max_states(
    orchard_prism, phi_states, psi_states
)[0]

# SCIP is an LP solver
from pyscipopt import Model
m = Model()

# Create a variable for each state
num_states = orchard_prism.nr_states
state_vars = [m.addVar(f"x_{i}", lb=0, ub=1) for i in range(num_states)]

# Encode LP
for state in range(num_states):
    if psi_states.get(state): # Case 1
        m.addCons(state_vars[state] == 1)
    elif prob0max_states.get(state): # Case 2
        m.addCons(state_vars[state] == 0)
    else: # Case 3
        for row in orchard_prism.transition_matrix.get_rows_for_group(state):
            summed_prob = 0
            for transition in orchard_prism.transition_matrix.get_row(row):
                prob = transition.value()
                next_state = transition.column
                summed_prob += prob * state_vars[next_state]
            m.addCons(state_vars[state] >= summed_prob)

In [19]:
# Solve LP
m.setObjective(sum(state_vars), sense="minimize")
m.optimize()
sol = m.getBestSol()
result_lp = sol[state_vars[orchard_prism.initial_states[0]]]

# Compare with default VI
properties = stormpy.parse_properties_without_context(
    'Pmax=? [F "PlayersWon"]'
)
vi_result = stormpy.model_checking(orchard_prism, properties[0].raw_formula)
result_vi = vi_result.at(orchard_prism.initial_states[0])

print(result_lp, result_vi)
assert abs(result_lp - result_vi) < 1e-6 # => True

presolving:
0.6313573066006342 0.6313566764945007
(round 1, fast)       16318 del vars, 16318 del conss, 0 add conss, 1253 chg bounds, 0 chg sides, 0 chg coeffs, 0 upgd conss, 0 impls, 0 clqs, 0 implints
(round 2, fast)       16347 del vars, 16351 del conss, 0 add conss, 1471 chg bounds, 0 chg sides, 0 chg coeffs, 0 upgd conss, 0 impls, 0 clqs, 0 implints
(round 3, fast)       16383 del vars, 16487 del conss, 0 add conss, 2510 chg bounds, 0 chg sides, 0 chg coeffs, 0 upgd conss, 0 impls, 0 clqs, 0 implints
(round 4, fast)       16496 del vars, 16679 del conss, 0 add conss, 5251 chg bounds, 0 chg sides, 0 chg coeffs, 0 upgd conss, 0 impls, 0 clqs, 0 implints
(round 5, fast)       16624 del vars, 16979 del conss, 0 add conss, 9886 chg bounds, 0 chg sides, 0 chg coeffs, 0 upgd conss, 0 impls, 0 clqs, 0 implints
(round 6, fast)       16868 del vars, 17410 del conss, 0 add conss, 14995 chg bounds, 0 chg sides, 0 chg coeffs, 0 upgd conss, 0 impls, 0 clqs, 0 implints
(round 7, fast)       171